In [1]:
from selenium import webdriver
import time
from tqdm import tqdm_notebook
import pandas as pd
import glob as gb

## Crawl data

In [2]:
# generate Webdriver object - Chrome version 79.0.3945.130 (64 bit)
driver = webdriver.Chrome('driver/chromedriver.exe')  # webdriver.Chrome('directory')

In [3]:
driver.get('http://www.opinet.co.kr')  # get('url')
driver.implicitly_wait(1)
driver.get('http://www.opinet.co.kr/searRgSelect.do')
driver.implicitly_wait(1)

In [4]:
# save option names list using id, <select> tag, attribute name
elem_sido = driver.find_element_by_id('SIDO_NM0')
elem_sido.send_keys('서울')
driver.implicitly_wait(1)

elem_sigungu = driver.find_element_by_id('SIGUNGU_NM0')  # find_element
gu_list = elem_sigungu.find_elements_by_tag_name('option')  # find_elements

gu_names = []
for gu in gu_list:
    tmp_val = gu.get_attribute('value')  # get_attribute( 'attr_name' )
    if tmp_val:  # if not null, append to list
        gu_names.append( tmp_val )
print(len(gu_names))
print(gu_names)

25
['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [5]:
# download only oil stations in Gang-nam gu (first elem in gu_list)
elem_sigungu.send_keys(gu_names[0])
driver.implicitly_wait(1)

driver.find_element_by_xpath('//*[@id="searRgSelect"]/span').click()
driver.implicitly_wait(7)
driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()

In [6]:
# download oil stations of each gu in gu_list
for name in tqdm_notebook(gu_names):
    elem_sigungu = driver.find_element_by_id('SIGUNGU_NM0')
    elem_sigungu.send_keys(name)
    driver.implicitly_wait(0.5)
    
    driver.find_element_by_xpath('//*[@id="searRgSelect"]/span').click()
    time.sleep(2)  # used time.sleep instead, due to errors
    driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
    pass

## Check the crawled data

In [7]:
filnames = gb.glob('data/지역*.xls')
print(len(filnames))  # crawled 26 when expected 25 (1 file redundant) → manually found 1 redundant file (using file size)

26


In [8]:
filnames = gb.glob('data/지역*[0-9]).xls')
print(len(gb.glob('data/지역*[0-9]).xls')))

25


In [9]:
df_list = []

for name in filnames:
    tmp_df = pd.read_excel(name, skiprows=2)
    df_list.append(tmp_df)
    pass

stations_raw = pd.concat(df_list, ignore_index=True)
print(stations_raw.shape)
stations_raw.tail()

(501, 10)


,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
496,서울특별시,노원주유소,서울 노원구 동일로 1008 (공릉동),GS칼텍스,02-949-5151,N,-,1639,1489,1100
497,서울특별시,하계삼호주유소,서울 노원구 공릉로 294 (하계동),GS칼텍스,02-974-8818,N,-,1660,1470,-
498,서울특별시,상계동주유소,서울 노원구 노원로26길 191 (상계동),현대오일뱅크,02-936-5035,N,-,1680,1540,1300
499,서울특별시,sk네트웍스(주)직영 하계주유소,서울 노원구 노원로17길 29 (하계동),SK에너지,02-975-1386,N,1995,1755,1599,1179
500,서울특별시,양지진흥개발(주)상계주유소,서울 노원구 노해로 527 (상계동),SK에너지,02-930-0111,N,-,1758,1598,1347


In [10]:
stations = pd.DataFrame({
    'name' : stations_raw['상호'],
    'address' : stations_raw['주소'],
    'brand' : stations_raw['상표'],
    'self_YN' : stations_raw['셀프여부'],
    'gas_price' : stations_raw['휘발유']
})

print(stations.shape)
stations.head()
stations.tail()

(501, 5)


,name,address,brand,self_YN,gas_price
496,노원주유소,서울 노원구 동일로 1008 (공릉동),GS칼텍스,N,1639
497,하계삼호주유소,서울 노원구 공릉로 294 (하계동),GS칼텍스,N,1660
498,상계동주유소,서울 노원구 노원로26길 191 (상계동),현대오일뱅크,N,1680
499,sk네트웍스(주)직영 하계주유소,서울 노원구 노원로17길 29 (하계동),SK에너지,N,1755
500,양지진흥개발(주)상계주유소,서울 노원구 노해로 527 (상계동),SK에너지,N,1758
